In [ ]:
import cv2
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
animal = 'DK78'
DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
CLEANED = os.path.join(DIR, 'CH1', 'thumbnail_cleaned')
ALIGNED = os.path.join(DIR, 'CH1', 'thumbnail_aligned')

In [ ]:
DK78_184_cleaned_path = os.path.join(CLEANED, '184.tif')
DK78_184_aligned_path = os.path.join(ALIGNED, '184.tif')
DK78_185_cleaned_path = os.path.join(CLEANED, '185.tif')
DK78_185_aligned_path = os.path.join(ALIGNED, '185.tif')

DK78_184_cleaned_img = cv2.imread(DK78_184_cleaned_path)
DK78_184_aligned_img = cv2.imread(DK78_184_aligned_path)
DK78_185_cleaned_img = cv2.imread(DK78_185_cleaned_path)
DK78_185_aligned_img = cv2.imread(DK78_185_aligned_path)

In [ ]:
orb = cv2.ORB_create(25)

img180_1 = cv2.imread('/home/eddyod/180.tif')
kp1 = orb.detect(img180_1,None)
kp1, des1 = orb.compute(img180_1, kp1)
blankimg180 = np.zeros(img180_1.shape).astype(np.uint8)
img180_2 = cv2.drawKeypoints(img180_1,kp1,None,color=(0,255,0), flags=0)

img181_1 = cv2.imread('/home/eddyod/181.tif')
kp2 = orb.detect(img181_1,None)
kp2, des2 = orb.compute(img181_1, kp2)
blankimg181 = np.zeros(img181_1.shape).astype(np.uint8)
img181_2 = cv2.drawKeypoints(img181_1,kp2,None,color=(0,255,0), flags=0)
#keypoints, descriptors = orb.detectAndCompute(img, None)

In [ ]:
blankimg180.shape

In [ ]:
bimg180 = cv2.drawKeypoints(blankimg180, kp1,None,color=(0,255,0), flags=0)
bimg181 = cv2.drawKeypoints(blankimg181, kp2,None,color=(0,255,0), flags=0)

In [ ]:
fig = plt.figure(figsize=(14., 14.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(2, 2),  # creates 2x2 grid of axes
                 axes_pad=0.5,  # pad between axes in inch.
                 )
for ax, im, title in zip(grid, [DK78_184_cleaned_img, DK78_184_aligned_img, 
                        DK78_185_cleaned_img, DK78_185_aligned_img],
                        ['184 unaligned', '184 aligned', '185 unaligned','185 aligned']):
    # Iterating over the grid returns the Axes.
    ax.set_title('Section {0} '.format(title))
    ax.imshow(im)

plt.show()